In [1]:
from sklearn.externals import joblib

In [2]:
with open("data/movie.pkl","rb") as f:
    data = joblib.load(f)
locals().update(data)

In [3]:
tdm

<30000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 122487 stored elements in Compressed Sparse Row format>

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tdm, sentiment, test_size=0.2, 
                                                    random_state=1234)

In [5]:
from sklearn.linear_model import LogisticRegressionCV

In [6]:
model = LogisticRegressionCV()

In [7]:
model.fit(x_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [8]:
model.score(x_train, y_train)

0.75429166666666669

In [9]:
model.score(x_test, y_test)

0.72833333333333339

In [10]:
model.coef_.shape

(1, 1000)

In [11]:
model.coef_[0,:10]

array([-0.96699223,  0.52531574, -0.17645564, -1.13258433,  0.66240135,
        0.22131067,  0.57946489,  0.93664739,  0.48513942,  0.24947403])

In [12]:
import pandas as pd

In [13]:
sent_df = pd.DataFrame({"단어": vectorizer.get_feature_names(), "계수": model.coef_.flat})
sent_df.tail()

C:\Users\Hyunsil\Anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:271: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


,단어,계수
995,youtube,-0.130874
996,yr,0.351412
997,yum,0.244320
998,yummy,0.918351
999,yup,0.466101


In [14]:
neg_df = sent_df[sent_df["계수"]<0].sort_values(by="계수")
neg_df.head()

,단어,계수
707,sad,-5.000790
551,miss,-3.204287
371,hate,-2.743394
750,sick,-2.642681
780,sorry,-2.601964


In [15]:
pos_df = sent_df[sent_df["계수"]>0].sort_values(by="계수", ascending=False)
pos_df.head()

,단어,계수
837,thanks,2.919784
836,thank,2.727561
348,great,2.262597
941,welcome,2.199017
43,awesome,2.149099


In [16]:
import numpy as np

In [17]:
sent_df["극성"] = np.sign(sent_df["계수"])
sent_df.set_index("단어", inplace=True)
sent_df.tail()

,계수,극성
단어,,
youtube,-0.130874,-1.0
yr,0.351412,1.0
yum,0.244320,1.0
yummy,0.918351,1.0
yup,0.466101,1.0


In [18]:
sentence = pd.read_csv("data/movie_test.csv")
sentence

,Test_Sentences
0,poor screen but reasonable price
1,very sad sorry for your mum
2,it's a lovely rainy day
3,really lol


In [19]:
pol = []
for sen in sentence["Test_Sentences"].values:
    words = sen.split()
    sent_df.reindex(words)
    sent = sent_df.reindex(words).dropna()
    pol.append([sen,(sent["극성"].sum() / sent["극성"].abs().sum())])

In [20]:
pol

[['poor screen but reasonable price', -1.0],
 ['very sad sorry for your mum', -1.0],
 ["it's a lovely rainy day", 0.33333333333333331],
 ['really lol', 0.0]]